In [35]:
# Import general libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [36]:
# Run to turn ON interactive visualization
%matplotlib tk

In [37]:
# Run to turn OFF interactive visualization
%matplotlib inline

In [38]:
# Function for Float32 to Uint8 conversion
def float32_to_uint8(float_img):
    # Check the current range of the image
    min_val = float_img.min()
    max_val = float_img.max()

    # Method 1: Simple scaling (for images already in 0-1 range)
    if 0 <= min_val and max_val <= 1:
        uint8_img = (float_img * 255).clip(0, 255).astype(np.uint8)

    # Method 2: Full normalization (for images with arbitrary ranges)
    else:
        # Normalize to 0-1 range first
        normalized_img = (float_img - min_val) / (max_val - min_val)
        # Then scale to 0-255
        uint8_img = (normalized_img * 255).clip(0, 255).astype(np.uint8)

    return uint8_img


In [39]:
# Load the image data: loaded as a 2D numpy array
# dtype='>f4': 32 bit little-endian single-precision float
from astropy.io import fits
from auto_stretch import apply_stretch

# Load the image data
img_path = 'dataset/hst_skycell-p0035x15y12_wfc3_ir_f160w_coarse-all_drz.fits'
raw_img = fits.getdata(img_path)

# Stretch the image for better visualization
stretched_img = apply_stretch(raw_img)
stretched_img_uint = float32_to_uint8(stretched_img)
cv2.imwrite('./outputs_lorenzo/original_image.png', stretched_img_uint)

True

In [40]:
sigma = 1.5
k = 5

blurred_image = cv2.GaussianBlur(stretched_img, (k, k), sigmaX=sigma)
blurred_image_uint = float32_to_uint8(blurred_image)
cv2.imwrite('./outputs_lorenzo/blurred_image.png', blurred_image_uint)

True

In [41]:
# k = 13 # best so far
k = 13
sobel_x = cv2.Sobel(stretched_img, cv2.CV_32F, 1, 0, ksize=k)
sobel_y = cv2.Sobel(stretched_img, cv2.CV_32F, 0, 1, ksize=k)
grad_mag = cv2.magnitude(sobel_x, sobel_y)
grad_mag_uint = float32_to_uint8(grad_mag)

grad_mag.max()

np.float32(1029788.3)

In [42]:
T = np.percentile(grad_mag, 99)
print(T)
_, star_mask = cv2.threshold(grad_mag_uint, T, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

star_mask_uint = float32_to_uint8(star_mask)
cv2.imwrite('./outputs_lorenzo/star_mask.png', star_mask)

121357.03


True

In [43]:
from scipy import ndimage

# Create true-false binary mask
bool_star_mask = star_mask > 0
bool_filled_mask = ndimage.binary_fill_holes(bool_star_mask)

# Convert back to uint8 with 0 and 255 values
filled_mask_uint = np.where(bool_filled_mask, 255, 0).astype(np.uint8)

cv2.imwrite('./outputs_lorenzo/filled_star_mask.png', filled_mask_uint)

True